In [ ]:
本文档禁止ai修改

## 📊 实验汇总

### ARC-Challenge-test集 测试结果详细记录

Llama-3.2-3B(source) 使用ARC-Challenge-train,valid Lora 微调，迁移至Qwen2.5 1.5B(target)
| 模型                        | 准确率(ACC)   | 提升   | Loss   | Perplexity | 微调/迁移配置                     | Run ID          | 备注         |
| --------------------------- | -------- | ------ | ------ | ---------- | ---------------------------- | --------------- | ------------ |
| Llama-3.2-3B (base)         | 0.6838   | -      | -      | -          | -                            | -               | 源模型基准        | 
| Llama-3.2-3B + LoRA         | 0.7082   | +3.57% | 0.1193 | 1.1275     | steps=600, batch=6, lr=1.5e-5 | 20250724_140508 |              | 
| Qwen2.5-1.5B (base)         | 0.7338   | -      | -      | -          | -                            | -               | 目标模型基准   |
| Qwen2.5-1.5B + 从llama迁移的7082 | 0.7372   | +0.34% | 0.7372 | 16679      | -                            | -               |              |
| Qwen2.5-1.5B + LoRA         | 0.7457   | +1.19% | 0.1193 | 1.1275     | steps=600, batch=6, lr=1.5e-5 | 20250724_014727 |              |




## 📊 实验汇总

### ARC-Challenge测试结果详细记录
Qwen2.5 1.5B(source) 使用ARC-Challenge-train,valid Lora 微调，迁移至Llama-3.2-3B(target)
| 模型                        | 准确率   | 提升   | Loss   | Perplexity | 训练配置                     | Run ID          | 备注         |
| --------------------------- | -------- | ------ | ------ | ---------- | ---------------------------- | --------------- | ------------ |
| Qwen2.5-1.5B (base)         | 0.7338   | -      | -      | -          | -                            | -               | 源模型基准   |
| Qwen2.5-1.5B + LoRA         | 0.7457   | +1.19% | 0.1193 | 1.1275     | steps=600, batch=6, lr=1.5e-5 | 20250724_014727 |              |
| Qwen2.5-1.5B + 从llama迁移的7082 | 0.7372   | +0.34% | 0.7372 | 16679      | -                            | -               |              |
| Llama-3.2-3B (base)         | 0.6838   | -      | -      | -          | -                            | -               | 目标模型基准 |
| Llama-3.2-3B + Qwen的LoRA   | 0.7065   | +2.27% | -      | -          | -                            | -               |              |
| Llama-3.2-3B + LoRA         | 0.7082   | +3.57% | 0.1193 | 1.1275     | steps=600, batch=6, lr=1.5e-5 | 20250724_140508 |              | 


In [ ]:
微调lora

cd /root/PAW/train_lora && python train_cs_lora_lightning.py --dataset arc-challenge --base_model /root/autodl-tmp/models/Llama-3.2-3B-Instruct --bs 4 --max_steps 600

In [ ]:
迁移

qwen 迁移到 llama
cd /root/PAW/lora_adapter && python scripts/transfer_lora_x.py   --source_lora /root/PAW/train_lora/runs/Qwen_Qwen2.5-1.5B/arc-challenge_lora_20250724_014727/final_model   --source_model /root/autodl-tmp/models/Qwen_Qwen2.5-1.5B   --target_model /root/autodl-tmp/models/Llama-3.2-3B-Instruct   --output /root/autodl-tmp/trained_t2l/qwen_to_llama_lora_x_fixed   --similarity_threshold 0.0001

llama 迁移到 qwen
cd /root/PAW/lora_adapter && python scripts/transfer_lora_x.py   --source_lora /root/PAW/train_lora/runs/Llama-3.2-3B-Instruct/arc-challenge_lora_20250724_140508/final_model   --source_model /root/autodl-tmp/models/Llama-3.2-3B-Instruct   --target_model //root/autodl-tmp/models/Qwen_Qwen2.5-1.5B   --output /root/autodl-tmp/lora-Xed/llama_to_qwen   --similarity_threshold 0.0001

In [ ]:
eval 迁移过的

qwen 迁移到 llama
python eval/lightning_eval.py --models_list /root/autodl-tmp/trained_t2l/qwen_to_llama_lora_x_fixed --dataset arc-challenge --sample_ratio 0.1 --base_model /root/autodl-tmp/models/Llama-3.2-3B-Instruct

llama 迁移到 qwen
python eval/lightning_eval.py --models_list /root/autodl-tmp/lora-Xed/llama_to_qwen --dataset arc-challenge --sample_ratio 0.1 --base_model /root/autodl-tmp/models/Qwen_Qwen2.5-1.5B

In [ ]:
eval base model

qwen
python eval/lightning_eval.py --models_list /root/autodl-tmp/lora-Xed/llama_to_qwen --dataset arc-challenge --sample_ratio 0.1 --base_model /root/autodl-tmp/models/Qwen_Qwen2.5-1.5B

llama